## task01 : 完成对数据集KMNIST的训练，代码入下

In [ ]:
#导入包
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import ToTensor #数据转换为张量
from torchvision.datasets import KMNIST #该数据集与fasionminist类似，形状和数量均是
from torch.utils.data import DataLoader #数据加载器
import sympy

In [70]:
#数据加载
train_data = KMNIST(root='./mydata2',train=True,download=True,transform=ToTensor())#训练集
test_data = KMNIST(root='./mydata2',train=False,download=True,transform=ToTensor())#测试集


In [ ]:
##定义超参数
LR = 0.8
epoachs = 20
BATCH_SIZE = 128#批次处理的样本个数

In [83]:
#分批次处理图片数据并打乱
train_dl =DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True)#shuffle是随机打乱数据

In [ ]:
#定义模型
model = nn.Sequential(
    nn.Linear(784,256),#输入层到第一个隐藏层
    nn.Sigmoid(),#激活函数
    nn.Linear(64,10)#第二个隐藏层，到输出层
)

In [85]:
# 损失函数
loss_function = nn.CrossEntropyLoss()  # 交叉熵损失函数
# 优化器（模型参数更新）
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

In [90]:
#模型训练

for epoach in range(epoachs):
        #提取训练数据 
    for data , target in train_dl :
         #前向运算
        output = model(data.reshape(-1,784))
         #计算损失
        loss = loss_function(output, target)
        # 反向传播
        optimizer.zero_grad() ##所有参数梯度清零
        loss.backward() ## 计算梯度（保存在参数.grad中）
        optimizer.step()#更新参数
    print(f"Epoch:{epoach}  Loss: {loss.item()}")

Epoch:0  Loss: 0.005807897541671991
Epoch:1  Loss: 0.004735148511826992
Epoch:2  Loss: 0.004878401290625334
Epoch:3  Loss: 0.00700853206217289
Epoch:4  Loss: 0.00556553527712822
Epoch:5  Loss: 0.004458623472601175
Epoch:6  Loss: 0.008920876309275627
Epoch:7  Loss: 0.005173067096620798
Epoch:8  Loss: 0.0064783780835568905
Epoch:9  Loss: 0.006512403022497892
Epoch:10  Loss: 0.0032078977674245834
Epoch:11  Loss: 0.005988527555018663
Epoch:12  Loss: 0.003826257772743702
Epoch:13  Loss: 0.006851695012301207
Epoch:14  Loss: 0.004051416181027889
Epoch:15  Loss: 0.00336522632278502
Epoch:16  Loss: 0.0012898811837658286
Epoch:17  Loss: 0.0031257374212145805
Epoch:18  Loss: 0.004958567675203085
Epoch:19  Loss: 0.002967755077406764


In [48]:
# test_data
# train_data

In [91]:
# 测试
test_dl = DataLoader(test_data, batch_size=BATCH_SIZE)

correct = 0
total = 0
with torch.no_grad():  # 不计算梯度
    for data, target in test_dl:
        output = model(data.reshape(-1, 784))#前向运算，输出后的形状为 [batch_size, num_classes]
        _, predicted = torch.max(output, 1)  # 返回每行最大值和索引，最大值不需要，用_省略，并且output中的索引就是图片类别，此处的1表示output的第一维度，也就是类别
        total += target.size(0)  # size(0) 等效 shape[0]，target.size(0) 返回 target 的第 0 维的大小，也就是当前批次中的样本数量。
        correct += (predicted == target).sum().item()

print(f'Accuracy: {correct/total*100}%')

Accuracy: 90.36999999999999%


## task 02 : 通过改变神经元数量和增加隐藏层来提升预测率，操作和结果如下

#### 初始隐藏层神经元数量为128，epoach=10,batch_size=128
相关代码：
model = nn.Sequential(
    nn.Linear(784,128),
    nn.Sigmoid(),
    nn.Linear(128,10)
)

运行结果：
        第一次Accuracy: 59.64%

        第二次Accuracy: 65.7% 

        第三次Accuracy: 67.84%
         
总结：可以看到，随着不断训练，Accuracy在不断升高，但是epoach较少，整体预测准确率不高

#### (1) 变更隐藏层神经元数量的尝试（初始隐藏层神经元数量为128，变更为256，epoach=10,batch_size=128）
相关代码：
        #定义模型
    model = nn.Sequential(

    nn.Linear(784,256),#输入层到隐藏层

    nn.Sigmoid(),#激活函数

    nn.Linear(256,10)#隐藏层，到输出层
)
运行结果：
        第一次Accuracy: 60.28%

        第二次Accuracy: 65.25999999999999%

        第三次Accuracy: 67.61%
总结：神经元数量变多，有助于初始准确率的提高，但随着训练的不断进行，准确率优化不高，并且隐藏层神经元数量越大，开销也越大

#### (2)变更隐藏层数量的尝试（初始为1，隐藏层数量变更为为2，epoach=10,batch_size=128）
相关代码：
    #定义模型
    
    model = nn.Sequential(
    nn.Linear(784,128),#输入层到第一个隐藏层

    nn.Sigmoid(),#激活函数

    nn.Linear(128,64),#第一个隐藏层到第二个隐藏层

    nn.Sigmoid(),

    nn.Linear(64,10)#第二个隐藏层，到输出层

)
运行结果：
        第一次Accuracy: 25.900000000000002%，降低

        第二次Accuracy: 40.08% ,降低

        第三次Accuracy: 53.7%，还是降低
         
总结：可以看到，隐藏层数量变多，损失增大，随着不断训练，Accuracy在不断升高，但是epoach较少，整体预测准确率较差
        


## task03 :改变epoach和lr的数值，观察准确率的变化


### 初始运行结果（初始模型为隐藏层神经元数量为256，隐藏层数量为1，lr=0.01，epochs=10）

运行结果：
        第一次Accuracy: 25.900000000000002%，降低

        第二次Accuracy: 40.08% ,降低

        第三次Accuracy: 53.7%，还是降低

#### (1)epochs由10改为50
结果：
第一次：Accuracy: 70.1%

第二次：Accuracy: 75.2%

第三次：Accuracy: 78.14%

总结：可以看到，伦次越多，准确率越高

#### (2)lr改为0.1，epoch=20

结果：
第一次Accuracy: 80.05%

第二次Accuracy: 85.2%

第三次Accuracy: 87.63%

总结：合适的学习率能够使模型更快的找到最优参数

#### (3)lr=0.8,epoach=20

第一次Accuracy: 90.07%

第二次Accuracy: 90.19%

第三次Accuracy: 90.36999999999999%

### 总结

适当的学习率和较多的轮次便于加快模型训练的速度，而隐藏层变多可能会导致开销变大，准确率较低，隐藏层神经元数量变多有利于准确率的升高，但也会增加开销